In [1]:
import os

import cv2

In [3]:
browser_name = "SequenceBrowser"
output_folder = r"J:\Temp"
image_name = "Video_Video"
name_prefix = "test_"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print("Created folder: " + export_fullpath)

In [4]:
browser_node = slicer.util.getFirstNodeByName(browser_name, className='vtkMRMLSequenceBrowserNode')
if browser_node is None:
    logging.error("Could not find browser node: {}".format(browser_name))
    raise

image_node = slicer.util.getFirstNodeByName(image_name)
if image_node is None:
    logging.error("Could not find image node: {}".format(image_name))
    raise

n = browser_node.GetNumberOfItems()

print("Sequence browser ID: {}".format(browser_node.GetID()))
print("Image node ID:       {}".format(image_node.GetID()))
print("Number of images:    {}".format(n))

Sequence browser ID: vtkMRMLSequenceBrowserNode1
Image node ID:       vtkMRMLStreamingVolumeNode1
Number of images:    912


In [15]:
browser_node.SelectFirstItem()

n = min(n, 10)
for i in range(n):
    image = image_node.GetImageData()
    shape = list(image.GetDimensions())
    shape.reverse()
    components = image.GetNumberOfScalarComponents()
    if components > 1:
        shape.append(components)
        shape.remove(1)
    image_array = vtk.util.numpy_support.vtk_to_numpy(image.GetPointData().GetScalars()).reshape(shape)
    image_name = name_prefix + "%04d" % i + ".png"
    image_fullname = os.path.join(output_folder, image_name)
    image_brg = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
    cv2.imwrite(image_fullname, image_brg)
    clear_output(wait = True)
    progress_text = ""
    for j in range(i):
        progress_text = progress_text + "="
    print(progress_text)
    browser_node.SelectNextItem()
    slicer.app.processEvents()

IntProgress(value=0, max=10)

=
==
===
====
=====


In [14]:
f = IntProgress(min=0, max=100)
display(f)

IntProgress(value=0)


In [12]:
f.value = 4